In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import rasterio.mask
from rasterio.features import rasterize
from rasterio.warp import reproject, Resampling
from rasterio.crs import CRS
from scipy.ndimage import distance_transform_edt

def align_to_dem_grid(tif_path, ref_crs, tr, out_shape, resampling=Resampling.bilinear):
    h, w = out_shape
    with rasterio.open(tif_path) as src:
        arr = src.read(1).astype("float32")
        if src.nodata is not None:
            arr = np.where(arr == src.nodata, np.nan, arr)
        if src.crs is None:
            raise ValueError(f"Missing CRS in raster: {tif_path}")
        if str(src.crs) != str(ref_crs):
            raise ValueError(f"CRS mismatch for {tif_path}. Need {ref_crs}, got {src.crs}")
        dst = np.full((h, w), np.nan, dtype="float32")
        reproject(
            source=arr,
            destination=dst,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=tr,
            dst_crs=ref_crs,
            resampling=resampling,
            src_nodata=np.nan,
            dst_nodata=np.nan,
        )
        return dst

def points_to_rc(points_xy, transform):
    rows, cols = [], []
    for x, y in points_xy:
        r, c = rasterio.transform.rowcol(transform, x, y)
        rows.append(r)
        cols.append(c)
    return np.array(rows, dtype=int), np.array(cols, dtype=int)

def clamp_rc(rows, cols, h, w):
    return np.clip(rows, 0, h - 1), np.clip(cols, 0, w - 1)

def build_distance_to_grid_m(lines_gdf, tr, out_shape, dx, dy):
    h, w = out_shape
    line_mask = rasterize(
        [(geom, 1) for geom in lines_gdf.geometry],
        out_shape=(h, w),
        transform=tr,
        fill=0,
        dtype="uint8"
    ).astype(bool)
    return distance_transform_edt(~line_mask, sampling=(dy, dx)).astype("float32")

def npv_from_series(capex0, annual_revenue, annual_opex, discount_rate, lifetime_years, degradation=0.0):
    years = np.arange(1, lifetime_years + 1, dtype="float64")
    energy_multiplier = (1.0 - degradation) ** (years - 1.0)
    cash = (annual_revenue * energy_multiplier) - annual_opex
    disc = (1.0 + discount_rate) ** years
    return -capex0 + np.sum(cash / disc)

def lcoe_from_series(capex0, annual_energy_mwh, annual_opex, discount_rate, lifetime_years, degradation=0.0):
    years = np.arange(1, lifetime_years + 1, dtype="float64")
    energy_multiplier = (1.0 - degradation) ** (years - 1.0)
    disc = (1.0 + discount_rate) ** years
    pv_cost = capex0 + np.sum(annual_opex / disc)
    pv_energy = np.sum((annual_energy_mwh * energy_multiplier) / disc)
    return pv_cost / pv_energy

def compute_financials_table(
    sites_xy,
    cf_grid,
    dist_m_grid,
    tr,
    installed_capacity_mw,
    capex_install_gbp_per_kw,
    connection_cost_gbp_per_km,
    opex_gbp_per_mw_year,
    prices_gbp_per_mwh_series,
    lifetime_years=25,
    discount_rate=0.07,
    degradation=0.005
):
    h, w = cf_grid.shape
    rows, cols = points_to_rc(sites_xy, tr)
    rows, cols = clamp_rc(rows, cols, h, w)

    cf_vals = cf_grid[rows, cols].astype("float64")
    dist_km_vals = (dist_m_grid[rows, cols].astype("float64")) / 1000.0

    avg_price = float(np.mean(prices_gbp_per_mwh_series))

    annual_energy_mwh = installed_capacity_mw * 8760.0 * cf_vals
    yearly_revenue_gbp = annual_energy_mwh * avg_price

    capex_install_gbp = installed_capacity_mw * 1000.0 * capex_install_gbp_per_kw
    capex_connection_gbp = dist_km_vals * connection_cost_gbp_per_km
    capex_total_gbp = capex_install_gbp + capex_connection_gbp

    annual_opex_gbp = installed_capacity_mw * opex_gbp_per_mw_year

    npv_vals = np.array([
        npv_from_series(
            capex0=float(capex_total_gbp[i]),
            annual_revenue=float(yearly_revenue_gbp[i]),
            annual_opex=float(annual_opex_gbp),
            discount_rate=discount_rate,
            lifetime_years=lifetime_years,
            degradation=degradation
        )
        for i in range(len(sites_xy))
    ], dtype="float64")

    lcoe_vals = np.array([
        lcoe_from_series(
            capex0=float(capex_total_gbp[i]),
            annual_energy_mwh=float(annual_energy_mwh[i]),
            annual_opex=float(annual_opex_gbp),
            discount_rate=discount_rate,
            lifetime_years=lifetime_years,
            degradation=degradation
        )
        for i in range(len(sites_xy))
    ], dtype="float64")

    df = pd.DataFrame({
        "site_id": [f"Farm_{i+1:02d}" for i in range(len(sites_xy))],
        "x": [p[0] for p in sites_xy],
        "y": [p[1] for p in sites_xy],
        "capacity_mw": installed_capacity_mw,
        "capacity_factor": cf_vals,
        "dist_to_grid_km": dist_km_vals,
        "capex_total_gbp": capex_total_gbp,
        "annual_opex_gbp": annual_opex_gbp,
        "yearly_revenue_gbp": yearly_revenue_gbp,
        "npv_gbp": npv_vals,
        "lcoe_gbp_per_mwh": lcoe_vals
    })

    return df

uk_boundary_27700_path = "/Users/lct/Desktop/0093/cw2/2nd_Assignment-20251218 2/datasets/uk_boundary_27700.geojson"
dem_tif_path = "/Users/lct/Desktop/0093/cw2/2nd_Assignment-20251218 2/datasets/uk_dem_clipped_UK_EPSG27700.tif"
capacity_tif_27700 = "/Users/lct/Desktop/0093/cw2/2nd_Assignment-20251218 2/datasets/GBR_capacity-factor_IEC1_EPSG27700.tif"
lines_path_27700 = "/Users/lct/Desktop/0093/cw2/2nd_Assignment-20251218 2/datasets/uk_transmission_lines_EPSG27700.geojson"

installed_capacity_mw = 500.0
capex_install_gbp_per_kw = 1588.0
connection_cost_gbp_per_km = 1_600_000.0
opex_gbp_per_mw_year = 23_100.0

prices_gbp_per_mwh_series = [
    207.07, 128.81,
    80.52,  86.02,
    83.50,  84.75
]

lifetime_years = 25
discount_rate = 0.07
degradation = 0.005

uk = gpd.read_file(uk_boundary_27700_path)
uk["geometry"] = uk.geometry.make_valid()
uk = uk[uk.geometry.notna() & ~uk.geometry.is_empty]
uk = uk.set_crs("EPSG:27700", allow_override=True)
uk_union = uk.union_all() if hasattr(uk, "union_all") else uk.unary_union

with rasterio.open(dem_tif_path) as src:
    dem_arr, tr = rasterio.mask.mask(src, [uk_union], crop=True, filled=True, nodata=np.nan)
    dem = dem_arr[0].astype("float32")
    if src.nodata is not None:
        dem = np.where(dem == src.nodata, np.nan, dem)
    ref_crs = src.crs

if ref_crs is None:
    ref_crs = CRS.from_epsg(27700)

h, w = dem.shape
dx = abs(tr.a)
dy = abs(tr.e)

cf_grid = align_to_dem_grid(capacity_tif_27700, ref_crs, tr, (h, w), resampling=Resampling.bilinear)

lines = gpd.read_file(lines_path_27700)
lines["geometry"] = lines.geometry.make_valid()
lines = lines[lines.geometry.notna() & ~lines.geometry.is_empty]

if lines.crs is None:
    lines = lines.set_crs(ref_crs, allow_override=True)
else:
    try:
        if str(lines.crs) != str(ref_crs):
            lines = lines.to_crs(ref_crs)
    except Exception:
        lines = lines.set_crs(ref_crs, allow_override=True)

dist_m_grid = build_distance_to_grid_m(lines, tr, (h, w), dx, dy)

try:
    sites_xy = [(float(x), float(y)) for (x, y, *_ ) in sites]
except Exception:
    sites_xy = [(float(p.x), float(p.y)) for p in sites_gdf.geometry]

results_df = compute_financials_table(
    sites_xy=sites_xy,
    cf_grid=cf_grid,
    dist_m_grid=dist_m_grid,
    tr=tr,
    installed_capacity_mw=installed_capacity_mw,
    capex_install_gbp_per_kw=capex_install_gbp_per_kw,
    connection_cost_gbp_per_km=connection_cost_gbp_per_km,
    opex_gbp_per_mw_year=opex_gbp_per_mw_year,
    prices_gbp_per_mwh_series=prices_gbp_per_mwh_series,
    lifetime_years=lifetime_years,
    discount_rate=discount_rate,
    degradation=degradation
)

results_df.to_csv("wind_financial_table_20_sites.csv", index=False)
results_df

summary_total = pd.DataFrame({
    "Metric": [
        "Number of sites",
        "Total installed capacity (MW)",
        "Total CAPEX (£)",
        "Total annual OPEX (£/yr)",
        "Total yearly revenue (£/yr)",
        "Mean CAPEX per site (£)",
        "Mean annual OPEX per site (£/yr)",
        "Mean yearly revenue per site (£/yr)",
        "Mean capacity factor",
        "Mean distance to grid (km)",
        "Mean NPV (£)",
        "Median NPV (£)",
        "Mean LCOE (£/MWh)",
        "Median LCOE (£/MWh)"
    ],
    "Value": [
        int(len(results_df)),
        float(results_df["capacity_mw"].sum()),
        float(results_df["capex_total_gbp"].sum()),
        float(results_df["annual_opex_gbp"].sum()),
        float(results_df["yearly_revenue_gbp"].sum()),
        float(results_df["capex_total_gbp"].mean()),
        float(results_df["annual_opex_gbp"].mean()),
        float(results_df["yearly_revenue_gbp"].mean()),
        float(results_df["capacity_factor"].mean()),
        float(results_df["dist_to_grid_km"].mean()),
        float(results_df["npv_gbp"].mean()),
        float(results_df["npv_gbp"].median()),
        float(results_df["lcoe_gbp_per_mwh"].mean()),
        float(results_df["lcoe_gbp_per_mwh"].median())
    ]
})

results_df["annual_net_cashflow_gbp"] = results_df["yearly_revenue_gbp"] - results_df["annual_opex_gbp"]
results_df["capex_share"] = results_df["capex_total_gbp"] / results_df["capex_total_gbp"].sum()
results_df["revenue_share"] = results_df["yearly_revenue_gbp"] / results_df["yearly_revenue_gbp"].sum()

site_rank = results_df[
    ["site_id", "capacity_factor", "dist_to_grid_km",
     "capex_total_gbp", "annual_opex_gbp", "yearly_revenue_gbp",
     "annual_net_cashflow_gbp", "npv_gbp", "lcoe_gbp_per_mwh",
     "capex_share", "revenue_share"]
].sort_values(["npv_gbp", "lcoe_gbp_per_mwh"], ascending=[False, True]).reset_index(drop=True)

summary_total.to_csv("wind_20sites_summary_total.csv", index=False)
site_rank.to_csv("wind_20sites_summary_by_site.csv", index=False)

summary_total, site_rank.head(10)



N = 25
r = 0.07
d = 0.005

years = np.arange(0, N + 1, dtype=float)
disc = (1.0 + r) ** years

capex0 = float(results_df["capex_total_gbp"].sum())
opex_annual = float(results_df["annual_opex_gbp"].sum())
revenue_annual_y1 = float(results_df["yearly_revenue_gbp"].sum())

energy_y1_mwh = float((results_df["capacity_mw"] * 8760.0 * results_df["capacity_factor"]).sum())

rev_t = np.zeros(N + 1, dtype=float)
opex_t = np.zeros(N + 1, dtype=float)
energy_t = np.zeros(N + 1, dtype=float)

for t in range(1, N + 1):
    mult = (1.0 - d) ** (t - 1)
    rev_t[t] = revenue_annual_y1 * mult
    opex_t[t] = opex_annual
    energy_t[t] = energy_y1_mwh * mult

net_cash_t = rev_t - opex_t
disc_net_cash_t = net_cash_t / disc
disc_energy_t = energy_t / disc

npv_annual = np.zeros(N + 1, dtype=float)
npv_annual[0] = -capex0
npv_annual[1:] = disc_net_cash_t[1:]

cum_npv = np.zeros(N + 1, dtype=float)
cum_npv[0] = -capex0
cum_npv[1:] = -capex0 + np.cumsum(disc_net_cash_t[1:])

running_lcoe = np.full(N + 1, np.nan, dtype=float)
pv_opex_cum = np.cumsum(opex_t / disc)
pv_energy_cum = np.cumsum(disc_energy_t)

for t in range(1, N + 1):
    pv_cost_t = capex0 + pv_opex_cum[t]
    pv_energy_t = pv_energy_cum[t]
    running_lcoe[t] = pv_cost_t / pv_energy_t if pv_energy_t > 0 else np.nan

plt.figure(figsize=(8, 5), dpi=300)
plt.scatter(years, npv_annual / 1e9, label="Annual discounted cash flow (Year 0 includes CAPEX)")
plt.scatter(years, cum_npv / 1e9, label="Cumulative NPV")
plt.axhline(0, linewidth=1)
plt.xlabel("Year")
plt.ylabel("£bn")
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 5), dpi=300)
plt.scatter(years[1:], running_lcoe[1:])
plt.xlabel("Year")
plt.ylabel("LCOE (£/MWh)")
plt.tight_layout()
plt.show()
